In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
import math
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from typing import Iterable, List
from tqdm.auto import tqdm
import datasets
import csv
from nltk.translate.bleu_score import sentence_bleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [4]:

print(validation_dataset[0])
print(len(validation_dataset))
print(test_dataset[0])
print(len(test_dataset))

{'translation': {'de': 'Die Premierminister Indiens und Japans trafen sich in Tokio.', 'en': 'India and Japan prime ministers meet in Tokyo'}}
2169
{'translation': {'de': 'Obama empfängt Netanyahu', 'en': 'Obama receives Netanyahu'}}
2999


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
modelName = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(modelName)
tokenizer = T5Tokenizer.from_pretrained(modelName)
prefix = "translate English to German: "

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import pipeline
transPipe = pipeline("translation_en_to_de", model="t5-small")



In [12]:
###### DO NOT RUN ###############
validationResults = transPipe([sentence["translation"]["en"] for sentence in validation_dataset])
testResults = transPipe([sentence["translation"]["en"] for sentence in test_dataset])


In [13]:
########## DO NOT RUN ####################
print("Validation Set Translations:")
for i in range(len(validation_dataset)):
    print("Source (English):", validation_dataset[i]["translation"]["en"])
    print("Translation (German):", validationResults[i]["translation_text"])

Validation Set Translations:
Source (English): India and Japan prime ministers meet in Tokyo
Translation (German): Die Premierminister Indiens und Japans treffen sich in Tokio
Source (English): India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.
Translation (German): Der neue indische Premierminister Narendra Modi trifft sein japanisches Amtskollege Shinzo Abe in Tokio, um über die wirtschaftlichen und sicherheitspolitischen Beziehungen zu diskutieren.
Source (English): Mr Modi is on a five-day trip to Japan to strengthen economic ties with the third largest economy in the world.
Translation (German): Herr Modi ist auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen zur drittgrößten Volkswirtschaft der Welt zu stärken.
Source (English): High on the agenda are plans for greater nuclear co-operation.
Translation (Ge

In [14]:
###### DO NOT RUN #############
print("Test Set Translations:")
for i in range(len(test_dataset)):
    print("Source (English):", test_dataset[i]["translation"]["en"])
    print("Translation (German):", testResults[i]["translation_text"])

Streaming output truncated to the last 5000 lines.
Source (English): On all of these measures, about a quarter of Democratic voters do not have an opinion of Sanders.
Translation (German): Bei all diesen Maßnahmen haben etwa ein Viertel der demokratischen Wähler keine Meinung von Sanders.
Source (English): As questions continue about Hillary Clinton's use of a personal email address and server while Secretary of State, most Democratic primary voters are satisfied with her explanation of the matter and say it hasn't impacted their overall views of her.
Translation (German): Während die Fragen über Hillary Clintons Verwendung einer persönlichen E-Mail-Adresse und eines Servers fortgesetzt werden, während Außenministerin, sind die meisten demokratischen Wählerinnen und Wähler zufrieden mit ihrer Erklärung der Angelegenheit und sagen, dass sie ihre allgemeine Meinung zu ihr nicht beeinflusst hat.
Source (English): But it's a different story among the American public overall.
Translation (G

In [18]:
############## DO NOT RUN #####################
import pickle
with open('/content/drive/MyDrive/lists.pkl', 'wb') as f:
    pickle.dump((validationResults, testResults), f)



In [8]:
import pickle
with open('/content/drive/MyDrive/lists.pkl', 'rb') as f:
    validationResults, testResults = pickle.load(f)

# Print loaded lists
print("List 1:", validationResults[2168]["translation_text"])
print("List 2:", testResults[2998]["translation_text"])

List 1: Im nächsten Jahr hofft er, dass Sicherheitsbarrieren für die Teilnehmer am Feldberg auf den Weg gebracht werden.
List 2: Foodwatch fordert die Verbraucher auf, gegen die EU-Pläne zu protestieren unter: www.quecksilber-aktion.foodwatch.de


In [9]:
##### FOR DEMO PURPOSES ###############
demoTest = transPipe(["My name is Rohan Gupta", "This is the NLP assignment 3."])
for i in range(len(demoTest)):
  print(demoTest[i]["translation_text"])

Mein Name ist Rohan Gupta
Dies ist die NLP-Zuweisung 3.


In [13]:
!pip install transformers datasets evaluate sacrebleu
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [22]:
########## CALCULATING EVALUATION METRICS #######################
references = []
predictions = []
for i in range(len(test_dataset)):
  references.append(test_dataset[i]["translation"]["de"])
  predictions.append(testResults[i]["translation_text"])

from evaluate import load
bertT = load("bertscore")
meteorT = load('meteor')
bleuT = load("bleu", max_order=1)
bleuT2 = load("bleu", max_order=2)
bleuT3 = load("bleu", max_order=3)
bleuT4 = load("bleu", max_order=4)

#bertR = bertT.compute(predictions=predictions, references=references, lang="de")
meteorR = meteorT.compute(predictions=predictions, references=references)
bleuR = bleuT.compute(predictions=predictions, references=references, max_order=1)
bleuR2 = bleuT2.compute(predictions=predictions, references=references, max_order=2)
bleuR3 = bleuT3.compute(predictions=predictions, references=references, max_order=3)
bleuR4 = bleuT4.compute(predictions=predictions, references=references, max_order=4)

#print("BertScore is: ", bertR)
print("Meteor Score is: ", meteorR)
print("Bleu-1 Score is: ", bleuR)
print("Bleu-2 Score is: ", bleuR2)
print("Bleu-3 Score is: ", bleuR3)
print("Bleu-4 Score is: ", bleuR4)

# bertR = bertT.compute(predictions=predictions, references=references, lang="de")
# print("BertScore is: ", bertR)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Meteor Score is:  {'meteor': 0.5904659873087972}
Bleu-1 Score is:  {'bleu': 0.6326077952348977, 'precisions': [0.6340460920882179], 'brevity_penalty': 0.997731557892608, 'length_ratio': 0.9977341269208061, 'translation_length': 62527, 'reference_length': 62669}
Bleu-2 Score is:  {'bleu': 0.49457010679249214, 'precisions': [0.6340460920882179, 0.3875319177529902], 'brevity_penalty': 0.997731557892608, 'length_ratio': 0.9977341269208061, 'translation_length': 62527, 'reference_length': 62669}
Bleu-3 Score is:  {'bleu': 0.3991115621266788, 'precisions': [0.6340460920882179, 0.3875319177529902, 0.2605034584018822], 'brevity_penalty': 0.997731557892608, 'length_ratio': 0.9977341269208061, 'translation_length': 62527, 'reference_length': 62669}
Bleu-4 Score is:  {'bleu': 0.32721372144381805, 'precisions': [0.6340460920882179, 0.3875319177529902, 0.2605034584018822, 0.1807296433975305], 'brevity_penalty': 0.997731557892608, 'length_ratio': 0.9977341269208061, 'translation_length': 62527, 'ref

In [26]:
import pandas as pd
print("########### CSV DEMO PIPELINE ################")
df = pd.read_csv("/content/drive/MyDrive/t5trial.csv")
english_sentences = df["en"].tolist()
german_sentences = [transPipe(sentence)[0]["translation_text"] for sentence in english_sentences]
translated_df = pd.DataFrame({"en": english_sentences, "de": german_sentences})
translated_df.to_csv("/content/drive/MyDrive/translated_sentences_partb.csv", index=False)

########### CSV DEMO PIPELINE ################
